### Preppin' Data Challenge 2023 Week 3 - Targets for DSB
<br>
Input the data

In [144]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('PD2023_WK01.csv')
df2 = pd.read_csv('PD2023_WK03_TARGETS.csv')

<br>
For the transactions file:
    
- Filter the transactions to just look at DSB 
    - These will be transactions that contain DSB in the Transaction Code field
- Rename the values in the Online or In-person field, Online of the 1 values and In-Person for the 2 values
- Change the date to be the quarter 
- Sum the transaction values for each quarter and for each Type of Transaction (Online or In-Person)

In [145]:
# Filtering to transactions 'DSB'

df1 = df1[df1['Transaction Code'].str.contains('DSB')]

In [146]:
# Renaming Online or In-person field
df1['Online or In-Person'] = np.where(df1['Online or In-Person'] == 1, 'Online', 'In-Person')

In [147]:
#Changing date to quarter
df1['Transaction Date'] = pd.to_datetime(df1['Transaction Date'], format = '%d/%m/%Y %H:%M:%S')
df1['Quarter'] = df1['Transaction Date'].dt.quarter

In [148]:
#Summing Transaction value for each quarter and 'Online or In-Person'

df1 = df1.groupby(['Quarter', 'Online or In-Person'], as_index=False)['Value'].sum()


<br>
For the targets file:
- Pivot the quarterly targets so we have a row for each Type of Transaction and each Quarter
 - Rename the fields
- Remove the 'Q' from the quarter field and make the data type numeric

In [149]:
# Unpivotting the dataframe using pd.melt()
df2 = pd.melt(df2, id_vars='Online or In-Person', var_name='Quarter', value_name = 'Quarterly Target')

In [150]:
#Removing 'Q' from the Quarter field
df2['Quarter'] = df2['Quarter'].str.replace('Q', '')

In [151]:
#Changing the datatype
df2['Quarter'] = df2['Quarter'].astype(int)

<br>
Join the two datasets together 
- You may need more than one join clause!
- Remove unnecessary fields

In [152]:
#Inner join on 'Online or In-Person', 'Quarter'
df = pd.merge(df1, df2, on=['Online or In-Person', 'Quarter'], how='inner')

<br>
Calculate the Variance to Target for each row

In [153]:
df['Variance'] = df['Value'] - df['Quarterly Target']

<br>
Output the Data

In [154]:
df.to_csv('PD2023W3_Output.csv', index=False)

In [155]:
#verifying the output
Output = pd.read_csv('PD2023W3_Output.csv')
Output

,Quarter,Online or In-Person,Value,Quarterly Target,Variance
0,1,In-Person,77576,75000,2576
1,1,Online,74562,72500,2062
2,2,In-Person,70634,70000,634
3,2,Online,69325,70000,-675
4,3,In-Person,74189,70000,4189
5,3,Online,59072,60000,-928
6,4,In-Person,43223,60000,-16777
7,4,Online,61908,60000,1908
